## Proyecto detección de lenguaje ofensivo

Es importante procesar las frases que queremos predecir de la misma manera que proceso el documento de entrenamiento para que el modelo puedan encontrar las mismas palabras.

In [1]:
#LIBRERIAS
import nltk
nltk.download('punkt')
from sklearn import svm
from nltk import TreebankWordTokenizer
from nltk.text import Text
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.text import Text
nltk.download('stopwords')
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle



ruta = "INSERTA_LA_RUTA_COMPLETA"




#CONECTAR CON CUENTA DE GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

#PARA CAMBIAR LA RUTA DEL DIRECTORIO DE ARCHIVOS DONDE ME ENCUENTRO
import os 
os.chdir(ruta)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
#CARGO EL TRAIN EN UNA LISTA PARA LUEGO DIVIDIRLO EN DOCUMENTS Y TAGS
train = pd.read_csv("corpus/train.tsv", sep='\t', header=0, encoding = "ISO-8859-1")

#CARGO EL LEXICON EN UNA LISTA
lexicon = pd.read_csv("corpus/SHARE.txt", sep='\t', encoding = "utf8")




#FUNCION PAR EL PREPROCESAMIENTO DEL TEXTO PASADO COMO ATRIBUTO
def procesar_texto(texto): 
  text_sinPunt = texto.translate(str.maketrans('', '', string.punctuation))

  #tokenizamos el contenido del texto
  tokenizer = TreebankWordTokenizer()
  tokens = tokenizer.tokenize(text_sinPunt)

  #eliminamos las palabras vacías
  stop_words = set(stopwords.words('spanish'))
  filtered_sentence = [w for w in tokens if not w.lower() in stop_words]

  #reducimos las palabras a su raíz
  palabras_stem=[]
  stemmer = SnowballStemmer("spanish")
  for token in filtered_sentence :
    palabras_stem.append(stemmer.stem(token))
  return " ".join(palabras_stem)


**UNA VEZ LEIDO EL EXCEL, EN EL CUADRO DE CÓDIGO SIGUIENTE SE DIVIDEN LAS TUPLAS EN LAS CASILLAS DE TEXTO Y ETIQUETAS, LAS ETIQUETAS LAS GUARDAMOS EN TAGS Y EL TEXTO EN DOCUMENTOS, COINCIDIENDO LAS POSICIONES DE CADA TEXTO CON SU ETIQUETA:**

In [ ]:
#RECORRO LAS TUPLAS DEL LEXICON PARA GUARDAR LA INFORMACION EN UNA LISTA
lexicon2 = lexicon.itertuples()
arrayLexicon = []
for fila in lexicon2:
  arrayLexicon.append(str(fila[1]))


#COMENTARIOS QUE SE VAN A USAR PARA ENTRENAR
documentos=[]
#ETIQUETAS DE DICHOS DOCUMENTOS
tags=[]
#ETIQUETAS DE GENERO DE DICHOS DOCUMENTOS
tags2=[]
#OBTENER LAS TUPLAS DEL ARCHIVO DE ENTRENAMIENTO
tuplas=train.itertuples()
contadorMujeres = 0
contadorHombres = 0

#RECORREMOS LAS TUPLAS Y LAS PROCESAMOS
for fila in tuplas:
  documentos.append(procesar_texto(str(fila[2])))
  tags.append(str(fila[3]))
  tags2.append(str(fila[4]))

**LOS DATOS YA HAN SIDO PREPROCESADOS, GUARDO EL CONTENIDO DE ENTRENAMIENTO EN TEXTO PLANO:**

In [ ]:
#GUARDADO DEL TEXTO PROCESADO Y DE LAS ETIQUETAS DEL CUNJUNTO DE ENTRENAMIENTO EN TEXTO PLANO:
with open("documentosProcesadosTrain.txt", 'w') as f:
  for doc in documentos:
    f.write(str(doc))
    f.write("\n")
with open("etiquetasProcesadasTrain.txt", 'w') as f:
  for tag in tags:
    f.write(str(tag))
    f.write("\n")

**USAMOS EL ALGORITMO DE CLASIFICACION MÁQUINAS DE VECTORES DE SOPORTES (SVM) PARA ENTRENAR EL MODELO:**

In [ ]:
#EXTRAIGO LAS CARACTERÍSTICAS USANDO TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documentos)

#COMBINO LAS DISTINTAS LISTAS DE ETIQUETAS DE ENTRENAMIENTO
y = np.vstack((tags, tags2)).T
svm = LinearSVC(random_state=42)

#CONVIERTO EL MODELO A ENTRENAR EN UN MULTILABEL CLASSIFIER ( CLASIFICADOR MULTI-ETIQUETA )
multilabel_classifier = MultiOutputClassifier(svm, n_jobs=-1)

#ENTRENAMOS EL MODELO CON EL CLASIFICADOR MULTI-ETIQUETA
multilabel_classifier = multilabel_classifier.fit(X, y)

**A PARTIR DE AQUI, AÑADIMOS LA PARTE DE TEST PARA HACER LA PREDICCIÓN SOBRE ÉL**

In [ ]:
#FUNCION PARA COMPROBAR SEGUN EL LEXICON, SI UN COMENTARIO ES OFENSIVO O NO SEGUN SI SOBREPASA UN UMBRAL
umbralLexicon = 3
vecesCambiadoPredic = 0

def procesarLexicon(pos):
  contador = 0
  for i in range(len(arrayLexicon)):
    if arrayLexicon[i] in textosProcesados[pos]:
      contador += 1
  if contador > umbralLexicon:
    return True
  return False




#DECLARAMOS UNOS TEXTOS PROCESADOS PARA HACER LAS PREDICCIONES Y OTROS SIN PROCESAR PARA MOSTRARLOS
textos = []
textosProcesados = []
comment_id_test = []
gender = []

#LEO EL ARCHIVO TEST.TSV
test = pd.read_csv("corpus/test.tsv", sep='\t', header=0, encoding = "utf8")
tuplas = test.itertuples()

#AQUI PROCESO EL TEXTO A PREDECIR GUARGANDO EN UN ARRAY PARA HACER LA PREDICCION
for fila in tuplas:
  textosProcesados.append(procesar_texto(str(fila[2])))
  textos.append(str(fila[2]))
  comment_id_test.append(str(fila[1]))
  gender.append(str(fila[3]))






#DECLARO UN ARRAY PARA GUARDAR LAS ETIQUETAS QUE SE CREARAN AL HACER LAS PREDICCIONES DE CADA FRASE
tagsGuardar = []

#AQUI COMIENZA LA PREDICCION
Y = vectorizer.transform(textosProcesados)
y_test_pred = multilabel_classifier.predict(Y)

with open("anotacionesTest.txt", 'w') as f:
  itPredicc=0
  mujeresOFF = 0
  hombresOFF = 0
  for text in textos:
    if "NON" in str(y_test_pred[itPredicc]):
      var = procesarLexicon(itPredicc)  #LE PASO EL INDICE PARA SABER QUE FRASE ES LA QUE ME HA SALIDO COMO NO OFENSIVA
      if var: #SI SE CUMPLE, ESQUE SE HA CAMBIADO LA ETIQUETA DE NON A OFF AL COMPROBAR CUANTAS PALABRAS OFENSIVAS CONTIENE SEGUN EL LEXICON
        vecesCambiadoPredic += 1
        tagsGuardar.append("OFF")
      else:
        tagsGuardar.append(str(y_test_pred[itPredicc][0]))
    else:
      tagsGuardar.append(str(y_test_pred[itPredicc][0]))
      if "woman" in str(y_test_pred[itPredicc][1]):
        mujeresOFF += 1
      else:
        hombresOFF += 1
    #print(text, "\n", y_test_pred[itPredicc][0], " ", y_test_pred[itPredicc][1])
    f.write(text)
    f.write("\n")
    f.write(y_test_pred[itPredicc][0])
    f.write(" ")
    f.write(y_test_pred[itPredicc][1])
    f.write("\n")
    itPredicc+=1


print("SE HA CAMBIADO LA PREDICCION DE ", vecesCambiadoPredic, " COMENTARIOS")
print("MUJERES A LAS QUE VA DIRIGIDO COMMENTS OFF: ", (mujeresOFF/len(gender))*100)
print("HOMBRES A LOS QUE VA DIRIGIDO COMMENTS OFF: ", (hombresOFF/len(gender))*100)

SE HA CAMBIADO LA PREDICCION DE  23  COMENTARIOS
MUJERES A LAS QUE VA DIRIGIDO COMMENTS OFF:  5.291783036895487
HOMBRES A LOS QUE VA DIRIGIDO COMMENTS OFF:  5.762163751286197


**PROCEDO A GUARDAR EL RESULTADO DE LA PREDICCION EN UN NUEVO ARCHIVO**

In [ ]:
#AQUI GUARDO EL RESULTADO DE LA PREDICICON EN UN ARCHIVO .TSV
data = {'comment_id': comment_id_test, 'pred': tagsGuardar}
df = pd.DataFrame(data, columns = ['comment_id', 'pred'])
df.to_csv('corpus/prediction_test.tsv', sep='\t', index=False)

**COMPRUEBO LA PRECISION DE MIS RESULTADOS CON EL CONJUNTO DE EVALUACIÓN gold_label_test**

In [ ]:
os.chdir("/content/drive/MyDrive/5ºInformática/2ºCUATRIMESTRE/PLN/Prácticas/Práctica 8/corpus")
!python scorer.py gold_label_test.tsv prediction_test.tsv
os.chdir("/content/drive/MyDrive/5ºInformática/2ºCUATRIMESTRE/PLN/Prácticas/Práctica 8")

GOLD_FILE: gold_label_test.tsv
PREDICTION_FILE: prediction_test.tsv
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` par